In [ ]:
import pandas as pd
import duckdb as dd
import sys
import os

carpeta = "TablasOriginales"

padron_poblacion =  pd.read_excel(carpeta + "/padron_poblacion.xlsX")
datos_departamento = pd.read_csv(carpeta + "/Datos_por_departamento_actividad_y_sexo.csv")
padron_establecimientos_educativos  = pd.read_excel(carpeta + "/2022_padron_oficial_establecimientos_educativos.xlsx", skiprows = 6)
actividades_establecimientos_productivos = pd.read_csv(carpeta + "/actividades_establecimientos.csv")



# Limpieza padron_poblacion


In [3]:

carpeta = "TablasOriginales"

padron_poblacion =  pd.read_excel(carpeta + "/padron_poblacion.xlsX")

padron_poblacion.columns = ['nada', 'edad', 'casos', 'porcentaje', 'porcentaje_acumulado']
padron_poblacion.columns


edad_index = []
total_index = []
areas = []
counter = 0
counter_total = 0
for i in range(len(padron_poblacion['edad'].tolist())): # saco los indices de fila donde aparece Edad 
    if padron_poblacion['edad'].tolist()[i] == 'Edad':
        counter += 1
        edad_index.append(i+1)
        areas.append(padron_poblacion['casos'].tolist()[i-2])


for i in range(len(padron_poblacion['edad'].tolist())): # saco los indices de fila donde aparece Total
    if padron_poblacion['edad'].tolist()[i] == 'Total':
        total_index.append(i)
        counter_total += 1

if  counter != counter_total:
    print("Error: Tamaños de indexacion diferentes")
    sys.exit()


df_list = []
for index in range(counter-1):
    edad = padron_poblacion['edad'][edad_index[index]: total_index[index]].tolist()
    casos = padron_poblacion['casos'][edad_index[index]: total_index[index]].tolist()
    porcentaje = padron_poblacion['porcentaje'][edad_index[index]: total_index[index]].tolist()
    porcentaje_acumulado = padron_poblacion['porcentaje_acumulado'][edad_index[index]: total_index[index]].tolist()
    
    data = {
        'edad': edad,
        'casos': casos,
        'porcentaje': porcentaje,
        'porcentaje_acumulado': porcentaje_acumulado
    }

    df_ = pd.DataFrame(data)

    ruta_subcarpeta = os.path.join('TablasModificadas', 'Padrones')

    os.makedirs(ruta_subcarpeta, exist_ok=True)

    ruta_archivo = os.path.join(ruta_subcarpeta, f'padron_poblacion_{areas[index]}.csv')
    df_.to_csv(ruta_archivo, index=False)
    
    df_list.append(df_)



# Limpieza Datos_departamento_por_actividad_y_genero

In [22]:
datos_departamento
consultaSQL = """
SELECT DISTINCT departamento
FROM datos_departamento 
              """

dataframeResultado = dd.sql(consultaSQL).df()
dataframeResultado

EP_deparatementos = list(dataframeResultado['departamento'])
conjunto1 = set(areas)
interseccion_set = conjunto1.intersection(EP_deparatementos)
interseccion_lista = list(interseccion_set)

print(len(interseccion_lista))  
print(len(areas))  
[x for x in EP_deparatementos if x not in interseccion_lista]

445
528


[]

In [5]:
print('1º de Mayo' == '1° de Mayo' )
print('Puan' == 'Puán')

False
False


In [ ]:
# Cambio el nombre en las columnas de '1º de Mayo' y 'Puan' en el df de EP

#saco los indices de los id sucios:
datos_departamento['departamento'] = datos_departamento['departamento'].replace({
    'Puán': 'Puan',
    '1° de Mayo': '1º de Mayo'
})
print(datos_departamento['departamento'].unique())




['Comuna 1' 'Comuna 2' 'Comuna 3' 'Comuna 4' 'Comuna 5' 'Comuna 6'
 'Comuna 7' 'Comuna 8' 'Comuna 9' 'Comuna 10' 'Comuna 11' 'Comuna 12'
 'Comuna 13' 'Comuna 14' 'Comuna 15' 'Adolfo Alsina'
 'Adolfo Gonzales Chaves' 'Alberti' 'Almirante Brown' 'Avellaneda'
 'Ayacucho' 'Azul' 'Bahía Blanca' 'Balcarce' 'Baradero' 'Arrecifes'
 'Benito Juárez' 'Berazategui' 'Berisso' 'Bolívar' 'Bragado' 'Brandsen'
 'Campana' 'Cañuelas' 'Capitán Sarmiento' 'Carlos Casares'
 'Carlos Tejedor' 'Carmen de Areco' 'Castelli' 'Colón'
 'Coronel de Marina Leonardo Rosales' 'Coronel Dorrego' 'Coronel Pringles'
 'Coronel Suárez' 'Chacabuco' 'Chascomús' 'Chivilcoy' 'Daireaux' 'Dolores'
 'Ensenada' 'Escobar' 'Esteban Echeverría' 'Exaltación de la Cruz'
 'Ezeiza' 'Florencio Varela' 'Florentino Ameghino' 'General Alvarado'
 'General Alvear' 'General Arenales' 'General Belgrano' 'General Guido'
 'General Juan Madariaga' 'General La Madrid' 'General Las Heras'
 'General Lavalle' 'General Paz' 'General Pinto' 'General Pueyrr

In [7]:
consultaSQL = """
SELECT DISTINCT departamento
FROM datos_departamento 
              """

dataframeResultado = dd.sql(consultaSQL).df()
dataframeResultado

EP_deparatementos = list(dataframeResultado['departamento'])
conjunto1 = set(areas)
interseccion_set = conjunto1.intersection(EP_deparatementos)
interseccion_lista = list(interseccion_set)

[x for x in EP_deparatementos if x not in interseccion_lista]

# todo joya

[]

In [8]:
# Me quedo con los datos que quiero:

consultaSQL = """
SELECT 
    departamento,
    clae6,
    SUM(Empleo) as empleados,
    SUM(Establecimientos) as establecimientos,
    SUM(empresas_exportadoras) as empresas_exportadoras
FROM datos_departamento
WHERE anio = 2022
GROUP BY clae6, departamento
ORDER BY departamento, empresas_exportadoras DESC
              """

datos_departamento_limpios = dd.sql(consultaSQL).df()
datos_departamento_limpios

,departamento,clae6,empleados,establecimientos,empresas_exportadoras
0,12 de Octubre,11501,21.0,7.0,6.0
1,12 de Octubre,461011,31.0,3.0,6.0
2,12 de Octubre,131110,215.0,7.0,4.0
3,12 de Octubre,11119,147.0,41.0,4.0
4,12 de Octubre,530010,1.0,1.0,1.0
...,...,...,...,...,...
89058,Ñorquín,472190,1.0,1.0,0.0
89059,Ñorquín,960990,1.0,1.0,0.0
89060,Ñorquín,471130,2.0,2.0,0.0
89061,Ñorquín,551022,7.0,3.0,0.0


In [9]:

ruta_subcarpeta = os.path.join('TablasModificadas', 'Datos_departamento')

os.makedirs(ruta_subcarpeta, exist_ok=True)


ruta_archivo = os.path.join(ruta_subcarpeta, 'Datos_departamento_por_actividad_y_sexo_limpios.csv')

datos_departamento_limpios.to_csv(ruta_archivo, index=False)

# Limpieza establecimientos educativos





In [10]:
# Dado que solo me importan los establecimientos educativos con relevancia para la produccion industrial entonces puedo sacar
# Todos aquellos que no entren en la categoria comun, o adultos de formacion profesional

In [11]:
padron_establecimientos_educativos  = pd.read_excel(carpeta + "/2022_padron_oficial_establecimientos_educativos.xlsx", skiprows = 6) # salto los irrelevantes
# saquemosle los espacios para poder llamarlos con sql despues:
padron_establecimientos_educativos.columns=['Jurisdicción', 'Cueanexo', 'Nombre', 'Sector', 'Ámbito', 'Domicilio',
       'C_P', 'Código_de_área', 'Teléfono', 'Código_de_localidad',
       'Localidad', 'Departamento', 'Mail', 'Común', 'Especial', 'Adultos',
       'Artística', 'Hospitalaria', 'Intercultural', 'Encierro',
       'Jardín_maternal', 'Jardín_de_infantes',
       'Primario', 'Secundario', 'Secundario_INET', 'SNU', 'SNU_INET',
       'Secundario.1', 'SNU.1', 'Talleres',
       'Nivel_inicial_Educación_temprana',
       'Nivel_inicial_Jardín_de_infantes.1', 'Primario.1', 'Secundario.2',
       'Integración_a_la_modalidad_común/_adultos', 'Primario.2',
       'Secundario.3', 'Alfabetización', 'Formación_Profesional',
       'Formación_Profesional_INET', 'Inicial', 'Primario.3', 'Secundario.4',
       'Unnamed: 43']
print(padron_establecimientos_educativos.columns)

Index(['Jurisdicción', 'Cueanexo', 'Nombre', 'Sector', 'Ámbito', 'Domicilio',
       'C_P', 'Código_de_área', 'Teléfono', 'Código_de_localidad', 'Localidad',
       'Departamento', 'Mail', 'Común', 'Especial', 'Adultos', 'Artística',
       'Hospitalaria', 'Intercultural', 'Encierro', 'Jardín_maternal',
       'Jardín_de_infantes', 'Primario', 'Secundario', 'Secundario_INET',
       'SNU', 'SNU_INET', 'Secundario.1', 'SNU.1', 'Talleres',
       'Nivel_inicial_Educación_temprana',
       'Nivel_inicial_Jardín_de_infantes.1', 'Primario.1', 'Secundario.2',
       'Integración_a_la_modalidad_común/_adultos', 'Primario.2',
       'Secundario.3', 'Alfabetización', 'Formación_Profesional',
       'Formación_Profesional_INET', 'Inicial', 'Primario.3', 'Secundario.4',
       'Unnamed: 43'],
      dtype='object')


In [12]:
consultaSQL = """
SELECT  Departamento,  Cueanexo, Sector, Nombre,Ámbito, Código_de_área ,Localidad, Código_de_localidad

FROM padron_establecimientos_educativos 


WHERE (Común == '1') OR (Formación_Profesional == '1') OR (Formación_Profesional_INET == '1')

              """

padron_establecimientos_educativos_limpio  = dd.sql(consultaSQL).df()
padron_establecimientos_educativos_limpio


,Departamento,Cueanexo,Sector,Nombre,Ámbito,Código_de_área,Localidad,Código_de_localidad
0,Comuna 4,20000100,Privado,INSTITUTO PRIVADO ESCUELA EVANGELICA WILLIAM C...,Urbano,11,CIUDAD DE BUENOS AIRES,2104001
1,Comuna 4,20000200,Estatal,CARLOS R. VIGNALE,Urbano,11,CIUDAD DE BUENOS AIRES,2104001
2,Comuna 13,20000300,Privado,INSTITUTO SAN CAYETANO,Urbano,11,CIUDAD DE BUENOS AIRES,2113001
3,Comuna 13,20000500,Privado,INSTITUTO NUESTRA SEÑORA DEL ROSARIO,Urbano,11,CIUDAD DE BUENOS AIRES,2113001
4,Comuna 9,20000700,Privado,INSTITUTO NTRA SEÑORA DE LUJAN DE LOS PATRIOTAS,Urbano,11,CIUDAD DE BUENOS AIRES,2109001
...,...,...,...,...,...,...,...,...
53275,USHUAIA,940021000,Estatal,COLEGIO PROVINCIAL DE EDUCACIÓN SECUNDARIA EVA...,Urbano,2901,USHUAIA,94014028
53276,RIO GRANDE,940021100,Estatal,ESCUELA MUNICIPAL EXPERIMENTAL LOS ÑIRES,Urbano,2901,TOLHUIN,94007153
53277,RIO GRANDE,940021500,Privado,Jardín de Infantes Semillitas,Urbano,2964,RIO GRANDE,94007152
53278,USHUAIA,940021600,Estatal,JARDIN DE INFANTES N°28,Urbano,2901,USHUAIA,94014028


In [13]:
tipos_educativos = [
    'Jardín_maternal',
    'Jardín_de_infantes', 
    'Primario',
    'Secundario',
    'Secundario_INET',
    'SNU',
    'SNU_INET',
    'Formación_Profesional',
    'Formación_Profesional_INET'
]

partes_sql = []

for tipo in tipos_educativos:
    parte = f"""
    SELECT Cueanexo, '{tipo}' as tipo_establecimiento
    FROM padron_establecimientos_educativos 
    WHERE (Común = '1' OR Formación_Profesional = '1' OR Formación_Profesional_INET = '1')
      AND {tipo} = '1'
    """
    partes_sql.append(parte)

consulta_union_completa = " UNION ALL ".join(partes_sql)

tabla_de_relaciones = dd.sql(consulta_union_completa).df()
tabla_de_relaciones

,Cueanexo,tipo_establecimiento
0,20000100,Jardín_maternal
1,20000300,Jardín_maternal
2,20000500,Jardín_maternal
3,20000700,Jardín_maternal
4,20001700,Jardín_maternal
...,...,...
70448,900214608,Formación_Profesional_INET
70449,900231300,Formación_Profesional_INET
70450,940014900,Formación_Profesional_INET
70451,940015300,Formación_Profesional_INET


In [ ]:
print(f"Total Cueanexos únicos: {padron_establecimientos_educativos_limpio['Cueanexo'].nunique()}")
print(f"Total filas: {len(tabla_de_relaciones)}")
#Ver que la tabla de relaciones es mayor que el padron de establecimientos limpio

Total Cueanexos únicos: 53280
Total filas: 70453


In [18]:
#Quiero guardar solo los datos de relevancia:

ruta_subcarpeta = os.path.join('TablasModificadas', 'Datos_EstablecimientosEducativos')

os.makedirs(ruta_subcarpeta, exist_ok=True)
ruta_archivo = os.path.join(ruta_subcarpeta, 'padron_establecimientos_educativos_limpio.csv')
padron_establecimientos_educativos_limpio.to_csv(ruta_archivo, index=False)

ruta_archivo = os.path.join(ruta_subcarpeta, 'tabla_de_relaciones_tiene_tipo.csv')
tabla_de_relaciones.to_csv(ruta_archivo, index=False)

# Actividades_departamentos no tiene limpieza asociada

In [19]:
ruta_subcarpeta = os.path.join('TablasModificadas', 'Actividades_establecimientos')

os.makedirs(ruta_subcarpeta, exist_ok=True)
ruta_archivo = os.path.join(ruta_subcarpeta, 'actividades_establecimientos_productivos.csv')


actividades_establecimientos_productivos.to_csv(ruta_archivo, index=False)


reviso que las id esten bien:

In [20]:
a = list(set(list(actividades_establecimientos_productivos['clae6'])))
b  = list(set(list(datos_departamento['clae6'])))

interseccion_set = list(set(a) & set(b))
interseccion_lista = list(interseccion_set)
[x for x in b if x not in interseccion_lista]
# No hay ids fuera

[]